# Oil Production

In [8]:
# DATA MANIPULATION, Visualization, and stats
import pandas as pd
from glob import glob
import os

In [21]:
# Your current working directory should be ./mattwagon/oil_production_prediction/notebooks
os.getcwd()

'/home/pte/code/mattwagon/oil_production_prediction/notebooks'

In [48]:
# This filepath should work for all of us
# I have added 'raw_data' to .gitignore so it should never be pushed to the repo
path = os.path.join('..', 'raw_data', 'WellA')

In [49]:
all_files = glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [50]:
df.dtypes.Date

dtype('O')

In [52]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'].dtype
df.Date

0      2014-01-01 00:00:00+01:00
1      2014-01-02 00:00:00+01:00
2      2014-01-03 00:00:00+01:00
3      2014-01-04 00:00:00+01:00
4      2014-01-05 00:00:00+01:00
                  ...           
5808   2019-12-27 00:00:00+01:00
5809   2019-12-28 00:00:00+01:00
5810   2019-12-29 00:00:00+01:00
5811   2019-12-30 00:00:00+01:00
5812   2019-12-31 00:00:00+01:00
Name: Date, Length: 5813, dtype: datetime64[ns, pytz.FixedOffset(60)]

In [46]:
df['Date'] = df['Date'].dt.tz_convert(None)

In [47]:
df.Date

0      2013-12-31 23:00:00
1      2014-01-01 23:00:00
2      2014-01-02 23:00:00
3      2014-01-03 23:00:00
4      2014-01-04 23:00:00
               ...        
5808   2019-12-26 23:00:00
5809   2019-12-27 23:00:00
5810   2019-12-28 23:00:00
5811   2019-12-29 23:00:00
5812   2019-12-30 23:00:00
Name: Date, Length: 5813, dtype: datetime64[ns]